In [ ]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 8.7MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
#import Library
import json
import pickle as pkl
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import gensim
import gensim.downloader as api
print(gensim.__version__)
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
3.8.3


#Data read

In [ ]:
#Data read from jason file
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
f = open('gdrive/My Drive/Abusive_text_detection/Preprocess/final_data.json') 
data = json.load(f)
f1 = open('gdrive/My Drive/Abusive_text_detection/Preprocess/class_value.json') 
label = json.load(f1)

In [ ]:
#split data as train and test
train_data, test_data, train_label, test_label = train_test_split(data,label, test_size=0.3, random_state=0)

In [ ]:
train_label = np.array(train_label)
test_label = np.array(test_label)

In [ ]:
#save label in google drive
with open('/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Label/train_label.pkl','wb') as f1,open('/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Label/test_label.pkl','wb') as f2:
  pkl.dump(train_label, f1)
  pkl.dump(test_label, f2)

#Taditional Machine Learning

#word2vec(Trainable)

In [ ]:
def sent2vec(words):
  M = []
  for w in words:
    try:
      M.append(embedding_matrix[w])
    except:
      continue
  M = np.array(M)
  v = M.sum(axis=0)
  if type(v) != np.ndarray:
    return np.zeros(300)        #here 300 is the embedding dimention
  return v / np.sqrt((v ** 2).sum())
  return v

In [ ]:
train_data_tokenize = [word_tokenize(x) for x in train_data]
test_data_tokenize = [word_tokenize(x) for x in test_data]

In [ ]:
model = gensim.models.Word2Vec(train_data_tokenize, size=300, min_count=1, workers=4)        #here 300 is the embedding dimention

In [ ]:
model.train(train_data_tokenize, total_examples=len(train_data_tokenize), epochs=4000)

(1364886920, 1421788000)

In [ ]:
embedding_matrix = dict(zip(model.wv.index2word,model.wv.syn0))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [ ]:
train_data_feature = [sent2vec(x) for x in tqdm(train_data_tokenize)]
test_data_feature = [sent2vec(x) for x in tqdm(test_data_tokenize)]

100%|██████████| 18271/18271 [00:00<00:00, 38933.48it/s]


In [ ]:
#save feature in google drive
with open('/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Extracted_feature_traditional_ML/Train/train_Feature_extraction_using_trainable_word2vec_Traditional_ML.pkl','wb') as f1,open('/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Extracted_feature_traditional_ML/Test/test_Feature_extraction_using_trainable_word2vec_Traditional_ML.pkl','wb') as f2:
  pkl.dump(train_data_feature, f1)
  pkl.dump(test_data_feature, f2)


In [ ]:
#Load data in google drive
# with open('/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Extracted_feature_traditional_ML/Feature_extraction_using_trainable_word2vec_Traditional_ML.pkl','rb') as f:
#   x = pkl.load(f)

# x[0] == data_feature[0]

#Pre-Trained Model

#Model selection catagory - wise
<pre>
1.   <b>Word2vec model :</b>
      -> 'word2vec-google-news-300'
2.   <b>Glove model :</b> 
     *  Twitter Glove Embeddings
        -> 'glove-twitter-25'
        -> 'glove-twitter-50'
        -> 'glove-twitter-100'
        -> 'glove-twitter-200'
     *  Wikipedia Glove Embeddings
        -> 'glove-wiki-gigaword-50'
        -> 'glove-wiki-gigaword-100'
        -> 'glove-wiki-gigaword-200'
        -> 'glove-wiki-gigaword-300'
3.   <b>FastText model :</b>
        -> 'fasttext-wiki-news-subwords-300'
</pre>

In [ ]:
def sent2vec_based_on_pretrained_embedding(words):
  M = []
  for w in words:
    try:
      M.append(word2vec_model[w])
    except:
      continue
  M = np.array(M)
  v = M.sum(axis=0)
  if type(v) != np.ndarray:
    return np.zeros(word2vec_model.vector_size)        #here word2vec_model.vector_size is the embedding dimention
  return v / np.sqrt((v ** 2).sum())

In [ ]:
#find feature and save feature in google drive
train_path = '/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Extracted_feature_traditional_ML/Train/train_Feature_extraction_using_'
test_path = '/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Extracted_feature_traditional_ML/Test/test_Feature_extraction_using_'
end_path = '_Traditional_ML.pkl'
middle_path = ['glove_twitter_25','glove_twitter_50','glove_twitter_100','glove_twitter_200','fasttext_wiki_news_subwords_300']
for i in middle_path:
  temp = i.replace("_", "-")
  print('working with ',temp)
  word2vec_model = api.load(temp)
  train_data_tokenize = [word_tokenize(x) for x in train_data]
  test_data_tokenize = [word_tokenize(x) for x in test_data]
  train_data_feature = [sent2vec_based_on_pretrained_embedding(x) for x in tqdm(train_data_tokenize)]
  test_data_feature = [sent2vec_based_on_pretrained_embedding(x) for x in tqdm(test_data_tokenize)]
  with open(train_path+i+end_path,'wb') as f1:
    pkl.dump(train_data_feature, f1)
  with open(test_path+i+end_path,'wb') as f2:
    pkl.dump(test_data_feature, f2)
  print('Created feature by ',temp,' upload finished')
  # print('Testing part for Created feature is okk or wrong : ')
  with open(train_path+i+end_path,'rb') as f1:
    x = pkl.load(f1)
  with open(test_path+i+end_path,'rb') as f2:
    y = pkl.load(f2)
  if (x[0].all() == train_data_feature[0].all()) and (y[0].all() == test_data_feature[0].all()):
    print('Created feature by ',temp,' is okk')
  else:
    print('Created feature by ',temp,' is wrong')

working with  glove-twitter-25


100%|██████████| 18271/18271 [00:00<00:00, 28113.64it/s]


Created feature by  glove-twitter-25  upload finished
Created feature by  glove-twitter-25  is okk
working with  glove-twitter-50
[==================================================] 100.0% 199.5/199.5MB downloaded


100%|██████████| 18271/18271 [00:01<00:00, 15537.54it/s]


Created feature by  glove-twitter-50  upload finished
Created feature by  glove-twitter-50  is okk
working with  glove-twitter-100
[==================================================] 100.0% 387.1/387.1MB downloaded


...

Created feature by  glove-twitter-100  upload finished
Created feature by  glove-twitter-100  is okk
working with  glove-twitter-200
[==============================================----] 94.0% 713.0/758.5MB downloaded


100%|██████████| 18271/18271 [00:00<00:00, 26440.86it/s]


Created feature by  glove-twitter-200  upload finished
Created feature by  glove-twitter-200  is okk
working with  fasttext-wiki-news-subwords-300
[=================================================-] 98.8% 946.6/958.4MB downloaded


100%|██████████| 18271/18271 [00:00<00:00, 25147.90it/s]


Created feature by  fasttext-wiki-news-subwords-300  upload finished
Created feature by  fasttext-wiki-news-subwords-300  is okk


#Deep Learning

#word2vec(Trainable)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#Deep Learning
#word2vec(Trainable)
vocab_size = 30000
# embedding_dim = 100
max_length = 250
# trunc_type='post'
oov_tok = "<OOV>"

# tokenizer = Tokenizer(num_words = vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(train_data)
training_padded = pad_sequences(training_sequences,maxlen=max_length) #truncating=trunc_type

testing_sequences = tokenizer.texts_to_sequences(test_data)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)# truncating=trunc_type

In [ ]:
total_words = len(word_index)
print(total_words)

25446


In [ ]:
#save trainning and testing feature
with open('/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Extracted_feature_deep_learning/training_padded.pkl','wb') as f1:
    pkl.dump(training_padded, f1)
with open('/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Extracted_feature_deep_learning/testing_padded.pkl','wb') as f1:
    pkl.dump(testing_padded, f1)
with open('/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Extracted_feature_deep_learning/vocab_size_and_max_length.pkl','wb') as f1:
    pkl.dump([vocab_size,max_length], f1)

#Pre-Trained Model

In [ ]:
#save embedding matrix
path = '/content/gdrive/My Drive/Abusive_text_detection/Feature_extraction/Extracted_feature_deep_learning/Embedding_matrix/embedding_matrix_using_' 
end_path = '_deep_learning.pkl'
middle_path = ['glove_twitter_25','glove_twitter_50','glove_twitter_100','glove_twitter_200','fasttext_wiki_news_subwords_300']
for i in middle_path:
  temp = i.replace("_", "-")
  print('working with ',temp)
  word2vec_model = api.load(temp)
  EMBEDDING_DIM = word2vec_model.vector_size
  embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
  for word, j in word_index.items():
    try:
      embedding_matrix[j] = word2vec_model[word]
    except:
      pass
  with open(path+i+end_path,'wb') as f1:
    pkl.dump(embedding_matrix, f1)
  print('Created embedding matrix by ',temp,' upload finished')
  # print('Testing part for Created feature is okk or wrong : ')
  with open(path+i+end_path,'rb') as f1:
    x = pkl.load(f1)
  if x.all() == embedding_matrix.all():
    print('Created embedding matrix by ',temp,' is okk')
  else:
    print('Created embedding matrix by ',temp,' is wrong')

working with  glove-twitter-25
Created embedding matrix by  glove-twitter-25  upload finished
Created embedding matrix by  glove-twitter-25  is okk
working with  glove-twitter-50
Created embedding matrix by  glove-twitter-50  upload finished
Created embedding matrix by  glove-twitter-50  is okk
working with  glove-twitter-100
Created embedding matrix by  glove-twitter-100  upload finished
Created embedding matrix by  glove-twitter-100  is okk
working with  glove-twitter-200
Created embedding matrix by  glove-twitter-200  upload finished
Created embedding matrix by  glove-twitter-200  is okk
working with  fasttext-wiki-news-subwords-300
Created embedding matrix by  fasttext-wiki-news-subwords-300  upload finished
Created embedding matrix by  fasttext-wiki-news-subwords-300  is okk
